# Exploratory Data Analysis
spiegare quali siano gli obbiettivi per i quali stiamo preparando i dataset



## Import Packages
* Il pacchetto **numpy** serve a lavorare con valori e array numerici. Performa efficienti conti matematici e statistici.
* Il pacchetto **pandas** serve a manipolare, pulire e gestire dati strutturati attraverso strutture come i DataFrame.
* Il pacchetto **geopandas** è un'estensione del pacchetto *pandas* e serve per gestire dati geospaziali e visualizzarli in mappe.

In [41]:
import pandas as pd
import numpy as np
import geopandas as gp

## Importa Data

Il primo DataFrame contiene i dati sui consumi delle linee elettriche di tutto il Trentino.  
Per ogni linea elettrica viene fornito il dato di consumo ogni 10 minuti per l'intero mese.  
I dati sono per il mese di Novembre 2013 (il processo verrà ripetuto per il mese di Dicembre 2013).


Il secondo DataFrame contiene i dati riguardanti le ubicazioni e linee elettriche presenti in ogni zona del Trentino.  
Per ogni zona del Trentino (identificata da *SQUAREID*) vengono associate le linee elettriche da lì passanti (*LINESET*) e per ogni combinazione di zona e linea elettrica viene associato il numero di ubicazioni presenti (*NR_UBI*)



In [42]:
set_df = pd.read_csv("./data/processed/SET-nov-2013.csv")
lin_df = pd.read_csv("./data/processed/line.csv")
set_df

,LINESET,DATE,POWER
0,DG1000420,01/11/2013 00:00,37.439999
1,DG1000420,01/11/2013 00:10,37.439999
2,DG1000420,01/11/2013 00:20,36.000000
3,DG1000420,01/11/2013 00:30,35.279999
4,DG1000420,01/11/2013 00:40,35.279999
...,...,...,...
794513,DG1056626,30/11/2013 23:10,46.549999
794514,DG1056626,30/11/2013 23:20,44.650002
794515,DG1056626,30/11/2013 23:30,44.599998
794516,DG1056626,30/11/2013 23:40,43.700001


Trasformiamo la colonna *DATE* in tre colonne separate che indicano il giorno (*DAY*), i minuti (*MIN*) e il giorno della settimana (*WEEK_DAY*).

In [43]:
set_df["DATE"] = pd.to_datetime(set_df["DATE"], format="%d/%m/%Y %H:%M", dayfirst=True)
set_df["DAY"] = pd.to_datetime(set_df["DATE"]).dt.day
set_df["MINUTE"] = pd.to_datetime(set_df["DATE"]).dt.hour * 60 + pd.to_datetime(set_df["DATE"]).dt.minute
set_df["WEEK_DAY"] = pd.to_datetime(set_df["DATE"]).dt.day_of_week
del set_df["DATE"]
set_df

,LINESET,POWER,DAY,MINUTE,WEEK_DAY
0,DG1000420,37.439999,1,0,4
1,DG1000420,37.439999,1,10,4
2,DG1000420,36.000000,1,20,4
3,DG1000420,35.279999,1,30,4
4,DG1000420,35.279999,1,40,4
...,...,...,...,...,...
794513,DG1056626,46.549999,30,1390,5
794514,DG1056626,44.650002,30,1400,5
794515,DG1056626,44.599998,30,1410,5
794516,DG1056626,43.700001,30,1420,5


Ora voglio associare i dati dei due DataFrame organizzarli in base alla zona

In [44]:
lin_df

,SQUAREID,LINESET,NR_UBI
0,155,DG1031221,4
1,272,DG1031221,72
2,273,DG1031221,21
3,276,DG1003117,34
4,277,DG1003117,3
...,...,...,...
2570,10983,DG1013504,5
2571,10984,DG1013504,2
2572,11097,DG1013503,1
2573,11098,DG1013503,6


Per prima cosa creo un DataFrame temporaneo dove ad ogni linea *LINESET* associo il numero di ubicazioni passanti per essa:

In [45]:
temp_df_line = lin_df.groupby("LINESET")["NR_UBI"].sum().reset_index()
temp_df_line = temp_df_line.rename(columns={"NR_UBI": "UBI_X_LINE"})
temp_df_line[:5]

,LINESET,UBI_X_LINE
0,DG1000011,6
1,DG1000023,62
2,DG1000420,2133
3,DG1000421,1424


Poi creo un DataFrame temporaneo dove ad ogni casella *SQUAREID* associo il numero di ubicazioni presenti:

In [46]:
temp_df_square = lin_df.groupby("SQUAREID")["NR_UBI"].sum().reset_index()
temp_df_square = temp_df_square.rename(columns={"NR_UBI": "UBI_X_SQUARE"})
temp_df_square[:5]

,SQUAREID,UBI_X_SQUARE
0,155,4
1,272,72
2,273,21
3,276,34


In [56]:
# set_df = pd.merge(temp_df_line, set_df, on="LINESET")
final_df1=pd.merge(lin_df,temp_df_line,on="LINESET")
final_df1[:5]

,SQUAREID,LINESET,NR_UBI,UBI_X_LINE
0,155,DG1031221,4,1380
1,272,DG1031221,72,1380
2,273,DG1031221,21,1380
3,389,DG1031221,65,1380


In [57]:

final_df2=pd.merge(lin_df,temp_df_square,on="SQUAREID")
final_df2[:5]

,SQUAREID,LINESET,NR_UBI,UBI_X_SQUARE
0,155,DG1031221,4,4
1,272,DG1031221,72,72
2,273,DG1031221,21,21
3,276,DG1003117,34,34


In [60]:
final_df = pd.merge(set_df,final_df1, on = "LINESET")
final_df[:5]

,LINESET,POWER,DAY,MINUTE,WEEK_DAY,SQUAREID,NR_UBI,UBI_X_LINE
0,DG1000420,37.439999,1,0,4,4037,2,2133
1,DG1000420,37.439999,1,0,4,4154,13,2133
2,DG1000420,37.439999,1,0,4,4155,20,2133
3,DG1000420,37.439999,1,0,4,4156,2,2133
4,DG1000420,37.439999,1,0,4,4269,1,2133


In [62]:

final_df = pd.merge(final_df,final_df2, on = "SQUAREID")
final_df#[:5]

,LINESET_x,POWER,DAY,MINUTE,WEEK_DAY,SQUAREID,NR_UBI_x,UBI_X_LINE,LINESET_y,NR_UBI_y,UBI_X_SQUARE_x,LINESET,NR_UBI,UBI_X_SQUARE_y
0,DG1000420,37.439999,1,0,4,4037,2,2133,DG1000420,2,2,DG1000420,2,2
1,DG1000420,37.439999,1,10,4,4037,2,2133,DG1000420,2,2,DG1000420,2,2
2,DG1000420,36.000000,1,20,4,4037,2,2133,DG1000420,2,2,DG1000420,2,2
3,DG1000420,35.279999,1,30,4,4037,2,2133,DG1000420,2,2,DG1000420,2,2
4,DG1000420,35.279999,1,40,4,4037,2,2133,DG1000420,2,2,DG1000420,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52006097,DG1056626,46.549999,30,1390,5,9089,8,1667,DG1056626,8,8,DG1056626,8,8
52006098,DG1056626,44.650002,30,1400,5,9089,8,1667,DG1056626,8,8,DG1056626,8,8
52006099,DG1056626,44.599998,30,1410,5,9089,8,1667,DG1056626,8,8,DG1056626,8,8
52006100,DG1056626,43.700001,30,1420,5,9089,8,1667,DG1056626,8,8,DG1056626,8,8


In [ ]:
nlines_square = lines_df.groupby("SQUAREID")["LINESET"].size().reset_index()
nlines_square = nlines_square.rename(columns={"LINESET": "NR_LINES"})